In [1]:
from prepare_data_for_modeling_child_or import ModelingData
import statsmodels.api as sm
from constants import *
import numpy as np
from sklearn.metrics import accuracy_score


In [2]:
obj = ModelingData()

data/BRFSS/2016/LLCP2016.XPT
data/BRFSS/2017/LLCP2017.XPT
data/BRFSS/2018/LLCP2018.XPT


In [4]:
mdf = obj.get_data_with_epa_region()

In [5]:
mdf['POVERTY_GUIDE'].sum()

3908017450.0

In [6]:
def fit_logistic(X, y):
    res = sm.Logit(y, X).fit()
    params = res.params
    conf = res.conf_int()
    conf['Odds Ratio'] = params
    conf.columns = ['5%', '95%', 'Odds Ratio']
    prediction = res.predict(X).apply(lambda x: 0 if x<0.5 else 1).tolist()
    print("Accuracy: ", accuracy_score(y,prediction))
    print(np.exp(conf))
    return conf, res

In [28]:
sc = MODELING_COLUMNS_FOR_CHILD + ['CHILDREN']

In [29]:
df = mdf[sc]

In [30]:
df

,TAILPIPE,RCSGENDR,_CPRACE,POVERTY,POPEST2017_CIV,ASTHMA,CHILDREN
0,0,2.0,1.0,0.0,0.093197,0,1.0
1,0,1.0,4.0,0.0,0.093197,0,1.0
2,0,2.0,1.0,0.0,0.093197,0,1.0
3,0,2.0,1.0,0.0,0.093197,1,1.0
4,0,2.0,1.0,1.0,0.093197,0,2.0
...,...,...,...,...,...,...,...
164566,0,2.0,1.0,1.0,NaN,0,2.0
164567,0,2.0,6.0,1.0,NaN,0,1.0
164568,0,1.0,1.0,1.0,NaN,0,1.0
164569,0,2.0,6.0,0.0,NaN,1,2.0


In [31]:
# keeping the same method for fillna
for column in df.columns:
    if df[column].isna().mean() > 0:
        print(column)
        df[column] = df[column].fillna(df[column].mode()[0])

_CPRACE
POVERTY
POPEST2017_CIV


c:\users\administrator\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


In [32]:
# A: just adding poverty/no income
X = df.drop(['ASTHMA'], axis=1)
y = df['ASTHMA']
conf, res = fit_logistic(X, y)

Optimization terminated successfully.
         Current function value: 0.293155
         Iterations 7
Accuracy:  0.9198886802656604
                      5%       95%  Odds Ratio
TAILPIPE        0.789237  0.860767    0.824226
RCSGENDR        0.354920  0.371723    0.363224
_CPRACE         0.996114  0.998875    0.997493
POVERTY         1.380226  1.513174    1.445172
POPEST2017_CIV  0.943405  0.971617    0.957407
CHILDREN        0.611330  0.632682    0.621914


In [33]:
df2 = mdf[sc]

In [34]:
df2.isna().mean()

TAILPIPE          0.000000
RCSGENDR          0.000000
_CPRACE           0.000036
POVERTY           0.095266
POPEST2017_CIV    0.020556
ASTHMA            0.000000
CHILDREN          0.000000
dtype: float64

In [35]:
df2 = df2.dropna()

In [36]:
# C: [dropping na]just adding poverty
X = df2.drop(['ASTHMA'], axis=1)
y = df2['ASTHMA']
conf, res = fit_logistic(X, y)

Optimization terminated successfully.
         Current function value: 0.290425
         Iterations 7
Accuracy:  0.9207108540254543
                      5%       95%  Odds Ratio
TAILPIPE        0.805682  0.883144    0.843524
RCSGENDR        0.342038  0.359447    0.350634
_CPRACE         0.996224  0.999327    0.997774
POVERTY         1.340882  1.478165    1.407851
POPEST2017_CIV  0.933245  0.963877    0.948438
CHILDREN        0.621489  0.644404    0.632843
